## Named Entity Recognition in the Trove Aboriginal Advocate

This data source is a XML format dump from the NLA Trove archive of one title - the Aboriginal Advocate.  The data is in the form of a large XML file contianing 3497 articles from this title.  The goal here is to run a named entity recognition process over the documents to extract names of interest. 

As with other notebooks in this project we will use the SpaCy language processing library to extract names from the text.  The first step is to define a reader for the XML data, this is done in the module [trovereader.py](trovereader.py) which is then imported here. 

In [ ]:
#!pip install -q -r requirements.txt

In [ ]:
import spacy
import csv
from urllib.request import urlopen
import os
import geocoder
import pandas as pd
import networkx as nx
import matplotlib.pylab as plt

%load_ext autoreload
# import these local modules and reload them if they change
%aimport trovereader
%aimport utils
%autoreload 1

In [ ]:
# the source XML filename - shared publicly on cloudstor
xmlurl = "https://cloudstor.aarnet.edu.au/plus/s/WEAzWn7kqORh8yD/download?path=%2F&files=nla.obj-573721295_Aborginies_Advocate.xml"

In [ ]:
# download the spacy model we need
model = 'en_core_web_lg'
#spacy.cli.download(model)
nlp = spacy.load(model)

The next cell uses the trove XML parser to read the separate document records in the XML file and run the NER system over these. The resulting entities are collected into a list of dictionaries which is then converted to a Pandas DataFrame.   We collect all entities that are found and for each one store a bit of context - the entity plus two tokens either side of it.  

Since finding entities can take some time, we write out the result to a CSV file.  We first check if the CSV file already exists and if it does, we read the entities from the file rather than recomputing them.  To force the NER process to run, set the variable `force` to `True` at the top of the cell.

In [ ]:
with urlopen(xmlurl) as xmlfile:
    df = trovereader.trove_to_dataframe(xmlfile)

print(df.shape)
df.head()

In [ ]:
# set to True if you want to recompute the entities even if the
# spreadsheet is present, False to read from the spreadsheet if it is there
force_ner = True

In [ ]:
csvname = "data/Aboriginies_Advocate_Entities.csv"

if not force_ner and os.path.exists(csvname):
    entities = pd.read_csv(csvname)
else:
    entities = utils.apply_ner(df, textcol='description', ident='identifier', context=5)
    entities.head()
    entities.to_csv(csvname, index=False)
    
entities.head(20)

Having extracted the entities we can now explore what we have found. Here we look at the locations (GPE) and oganisations (ORG) and see what the most frequent 30 entities are in each case. 

In [ ]:
locations = entities[entities.type == "GPE"]
locations.entity.value_counts()[:30]

In [ ]:
orgs = entities[entities.type == "ORG"]
orgs.entity.value_counts()[:30]

In [ ]:
mention_graph = nx.DiGraph()
mention_graph = nx.from_pandas_edgelist(entities, source='id', target='entity', edge_attr=True, create_using=mention_graph)


In [ ]:
mention_graph.number_of_nodes(), mention_graph.number_of_edges()

In [ ]:
result = []
for name, group in locations.groupby('id'):
    locs = group.entity
    for place1 in group.entity:
        for place2 in group.entity:
            if not place1 == place2:
                result.append({'loc1': place1, 'loc2': place2, 'doc': name})

colloc = pd.DataFrame(result).sort_values(['loc1','loc2'])
colloc.head()

In [ ]:
collocg = colloc.groupby(['loc1', 'loc2']).count()

In [ ]:
df = pd.DataFrame({
    'loc1': collocg.index.get_level_values('loc1'), 
    'loc2': collocg.index.get_level_values('loc2'), 
    'count': collocg.doc,
    'weight': [1/c for c in collocg.doc]   # weight is inverse of count
})
df.index = range(df.shape[0])  # reset index to integers
df.head()

In [ ]:
# select those collocations that occur more than 10 times
df10 = df[df['count'] > 10]
df10.head()

In [ ]:
cg = nx.from_pandas_edgelist(df10, source='loc1', target='loc2', edge_attr=True, create_using=nx.DiGraph())
cg.size()

In [ ]:
node = "Kingswood"
nodes = cg.neighbors(node)
nodes = list(nodes)
nodes.append(node)
subg = cg.subgraph(nodes)
print("Subgraph of nodes linked to", node, "contains", subg.size(), "nodes")

In [ ]:
plt.figure(figsize=(20,20))
pos = nx.kamada_kawai_layout(subg, weight="weight")
colours = [(n==node and'g') or 'y' for n in subg.nodes]
ecol = [e[2] for e in subg.edges.data('weight')]
ecol = 'grey'
nx.draw_networkx(subg, pos, edge_color=ecol, font_weight="bold", font_color='r', arrows=False, node_color=colours)

In [ ]:

df10[df10.loc1==node].sort_values('count', ascending=False)

In [ ]:
df10.to_csv("advocate-locations-collocation.csv")
locations.to_csv("advocate-locations.csv")

In [ ]:
locations[locations['entity'] == 'Kingswood'].head(20)